In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('IMDB Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df = df.sample(10000)

In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
negative,5010
positive,4990


In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
print(df.duplicated().sum())

20


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
print(df.duplicated().sum())

0


**Basic Preprocessing**
1. **Make text in lowercase**
2. **Remove Tags**
3. **Remove Punctuation Marks**
4. **Remove Stop Words**

In [ ]:
df['review'] = df['review'].str.lower()

In [ ]:
df['review']

,review
30495,this movie is brilliant in every way. it touch...
6448,finally we get a tv series where we get to see...
35971,relesed from troma (which is my favorite movie...
11932,my daughter already wrote a review of this mov...
46149,holy crap this is so hysterical! why aren't am...
...,...
36276,"i must confess, i was surprised at how good th..."
30691,as a forty-something urban explorer/photograph...
13582,"""death wish 3"" brings back charles bronson as ..."
4216,the limited scenery views were the only saving...


In [ ]:
import re
def remove_tags(text):
  pattern = re.compile('<.*?>')
  return pattern.sub(r'',text)

In [ ]:
df['review'] = df['review'].apply(remove_tags)

In [ ]:
df['review']

,review
30495,this movie is brilliant in every way. it touch...
6448,finally we get a tv series where we get to see...
35971,relesed from troma (which is my favorite movie...
11932,my daughter already wrote a review of this mov...
46149,holy crap this is so hysterical! why aren't am...
...,...
36276,"i must confess, i was surprised at how good th..."
30691,as a forty-something urban explorer/photograph...
13582,"""death wish 3"" brings back charles bronson as ..."
4216,the limited scenery views were the only saving...


In [ ]:
import string
exclude = string.punctuation
def remove_punc(text):
  for char in exclude:
    text = text.replace(char,'')
  return text

In [ ]:
df['review'] = df['review'].apply(remove_punc)

In [ ]:
df['review']

,review
30495,this movie is brilliant in every way it touche...
6448,finally we get a tv series where we get to see...
35971,relesed from troma which is my favorite movie ...
11932,my daughter already wrote a review of this mov...
46149,holy crap this is so hysterical why arent amer...
...,...
36276,i must confess i was surprised at how good thi...
30691,as a fortysomething urban explorerphotography ...
13582,death wish 3 brings back charles bronson as on...
4216,the limited scenery views were the only saving...


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_stopwords(text):
  new_text = []
  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)
  x = new_text[:]
  new_text.clear()
  return " ". join(x)


In [ ]:
df['review'] = df['review'].apply(remove_stopwords)

In [ ]:
df['review']

,review
30495,movie brilliant every way touches comple...
6448,finally get tv series get see acting tal...
35971,relesed troma favorite movie companyunspea...
11932,daughter already wrote review movie sign...
46149,holy crap hysterical arent american comedi...
...,...
36276,must confess surprised good movie fir...
30691,fortysomething urban explorerphotography lo...
13582,death wish 3 brings back charles bronson one ...
4216,limited scenery views saving grace other...


In [ ]:
x = df.iloc[:,0:1]
y = df['sentiment']

In [ ]:
x.shape

(9980, 1)

In [ ]:
y

,sentiment
30495,positive
6448,positive
35971,negative
11932,positive
46149,positive
...,...
36276,positive
30691,positive
13582,positive
4216,negative


In [ ]:
from sklearn.preprocessing import LabelEncoder
ohe = LabelEncoder()
y = ohe.fit_transform(y)

In [ ]:
y

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [ ]:
x_train.shape

(7984, 1)

**Applying Bag of Words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [ ]:
x_train_bow.shape

(7984, 72615)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(x_test_bow)

In [ ]:
y_pred

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_pred)

0.62374749498998

In [ ]:
confusion_matrix(y_test,y_pred)

array([[678, 316],
       [435, 567]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
rf.fit(x_train_bow, y_train)

RandomForestClassifier()

In [ ]:
pred = rf.predict(x_test_bow)

In [ ]:
accuracy_score(y_test,pred)

0.8376753507014028

In [ ]:
confusion_matrix(y_test,pred)

array([[851, 143],
       [181, 821]])

In [ ]:
cv = CountVectorizer(max_features=7000)
x_train_bow1 = cv.fit_transform(x_train['review']).toarray()
x_test_bow1 = cv.transform(x_test['review']).toarray()

In [ ]:
rf.fit(x_train_bow1, y_train)

RandomForestClassifier()

In [ ]:
pred1 = rf.predict(x_test_bow1)
accuracy_score(y_test,pred1)

0.8396793587174348

In [ ]:
cv = CountVectorizer(ngram_range=(1,2), max_features=7000)
x_train_bow2 = cv.fit_transform(x_train['review']).toarray()
x_test_bow2 = cv.transform(x_test['review']).toarray()
rf.fit(x_train_bow2, y_train)
pred2 = rf.predict(x_test_bow2)
accuracy_score(y_test,pred2)

0.8361723446893787

**using TfIdf Vectorizer**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfd = TfidfVectorizer(max_features=20000)
x_train_tfidf = tfd.fit_transform(x_train['review']).toarray()
x_test_tfidf = tfd.transform(x_test['review']).toarray()
rf.fit(x_train_tfidf,y_train)
pred3 = rf.predict(x_test_tfidf)
accuracy_score(y_test,pred3)

0.8212318477716575

**Applying Word2Vec**

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 19.3 MB/s eta 0:00:00


In [ ]:
import os
import gensim

In [ ]:
from gensim.models import word2vec, keyedvectors

In [ ]:
import gensim.downloader as api

model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
model['cricket'].shape

(300,)

In [ ]:
x_train = x_train['review'].apply(lambda x:" ".join(x))

In [ ]:
x_test = x_test['review'].apply(lambda x:" ".join(x))

In [ ]:
import spacy
import en_core_web_sm
# Load the spacy model. This takes a few seconds.
nlp = en_core_web_sm.load()
# Process a sentence using the model
doc = nlp(x_train.values[0])
print(doc.vector)

[ 0.09788851 -0.26167282  0.6280999   0.22008297 -0.28359368 -0.42687052
  0.12927417  0.85750246  0.54525197  0.02603221 -0.11925994 -0.09512832
 -0.18594952  0.34645614 -0.03676303  0.30931187 -0.09268291 -0.33814487
  0.11262847 -0.2959117   0.05692845  0.23223197 -0.4733222   0.09927595
  0.4628975  -0.31339923  0.1516453  -0.1037797   0.1416579   0.20903914
 -0.4908614   0.07753    -0.01508275  0.46494627 -0.30346796  0.03341379
  0.48983374  0.53788155 -0.13495047  0.21901508 -0.11031096  0.03520443
  0.87372863  0.07222562 -0.05982334  0.71125215 -0.22952734  0.19181882
 -0.26203674  0.09985501  0.3929095  -0.18670693 -0.15662365  0.1845746
 -0.48765633 -0.25383994 -0.04681338  0.10514599  0.15211725 -0.31746927
 -0.6003859  -0.104784    0.05098068 -1.0870544  -0.594035    0.1537857
  0.10764071  0.3461954   0.20012267 -0.36474907 -0.03893522  0.05551388
  0.8256392  -0.5517032   0.44537273 -0.33330506 -0.18213785  0.55401665
 -0.03412237 -0.01322854 -0.39313006 -0.27084976 -0.1

In [ ]:
import spacy
import en_core_web_sm
# Load the spacy model. This takes a few seconds.
nlp = en_core_web_sm.load()
# Process a sentence using the model
doc1 = nlp(x_test.values[0])
print(doc1.vector)

[ 0.05545218 -0.26392162  0.6093557   0.1696618  -0.2883902  -0.4469817
  0.14385816  0.8582906   0.54476935  0.02136761 -0.14970453 -0.13242535
 -0.1689479   0.2663685  -0.07300529  0.3180324  -0.09874231 -0.37914532
  0.16847993 -0.30522937  0.07694092  0.23826061 -0.45161644  0.08088999
  0.47684616 -0.36172286  0.16678931 -0.1382825   0.09861161  0.2306145
 -0.48437095  0.04166092 -0.02533854  0.49999142 -0.36826596  0.07699677
  0.47177282  0.50985    -0.19012201  0.27542892 -0.09062141  0.02486418
  0.79643756  0.1569617  -0.03636025  0.7104064  -0.27076316  0.18090494
 -0.18916094  0.09604129  0.43298957 -0.20290512 -0.11978027  0.13606302
 -0.40189815 -0.34168026 -0.00476162  0.08642067  0.17119619 -0.27896285
 -0.6310834  -0.07621191  0.06074319 -1.1054906  -0.609579    0.13228172
  0.11726388  0.43766895  0.22436158 -0.34083644 -0.03920326  0.04620843
  0.85446393 -0.6164184   0.3889132  -0.3180236  -0.21562403  0.5730212
 -0.02657701 -0.03499186 -0.40900436 -0.2972364  -0.06

In [ ]:
import numpy as np

In [ ]:
input_arr = []
for item in x_train.values:
    doc = nlp(item)
    input_arr.append(doc.vector)

In [ ]:
input_arr = np.array(input_arr)

In [ ]:
input_arr.shape

(7987, 96)

In [ ]:
input_test_arr = []
for item in x_test.values:
    doc = nlp(item)
    input_test_arr.append(doc.vector)

In [ ]:
input_test_arr = np.array(input_test_arr)

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()
gnb.fit(input_arr, y_train)
pred4= gnb.predict(input_test_arr)
accuracy_score(y_test,pred4)

0.5928893340010015

**Train word2vec on own dataset**

In [ ]:
from gensim.utils import simple_preprocess
from nltk import sent_tokenize

In [ ]:
x_train['review']

,review
13529,great little groundbreaking movie 1955 impo...
25214,feeling need someone play role arbiter ...
17517,movie absolutely pointless one good esampl...
25559,understand fox embarrassed released pg13...
31716,cliffhanger decent action crime adventure ...
...,...
28305,lot obvious hype associated film lets f...
30041,bought movie see bam really loving s...
37558,mistake stilted dialog andor dismiss arche...
41304,step aside hollywood veterans wayan brothers...


In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
allreviews = []
for doc in df['review']:
  raw_sent = sent_tokenize(doc)
  for sent in raw_sent:
    allreviews.append(simple_preprocess(sent))


In [ ]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count =2
)

In [ ]:
model.build_vocab(allreviews)

In [ ]:
model.train(allreviews, total_examples=model.corpus_count,epochs=model.epochs)

(5498581, 5930140)

In [ ]:
len(model.wv.index_to_key)

35312

In [ ]:
def remove_oov(text):
  text = [word for word in text.split() if word in model.wv.index_to_key]
  return np.mean(model.wv[text],axis=0)


In [ ]:
remove_oov(df['review'].values[0])

array([-3.13596785e-01,  5.93843162e-01,  8.47128630e-02, -7.49876872e-02,
        5.99066168e-03, -5.97489834e-01,  3.70680273e-01,  8.89328122e-01,
       -3.33735734e-01, -3.28930616e-01, -4.23450135e-02, -8.62238586e-01,
        4.03640233e-02,  2.34051451e-01,  1.99727938e-01, -4.28650379e-01,
       -1.74426422e-01, -4.86758143e-01, -2.37391010e-01, -1.08751523e+00,
        2.80952007e-01,  2.97674567e-01,  4.45537776e-01, -6.86053261e-02,
       -2.81059116e-01, -9.67835188e-02, -4.65696864e-02, -4.42711532e-01,
       -3.94105673e-01, -2.01032814e-02,  3.87424886e-01,  7.54530132e-02,
        2.84268856e-01, -6.20915174e-01, -1.35906100e-01,  4.00169164e-01,
        6.85982481e-02, -5.91216564e-01, -2.35666975e-01, -6.21338487e-01,
        2.72244122e-02, -4.36428428e-01, -1.44166082e-01,  4.78198566e-02,
        2.31442675e-01, -2.23596573e-01, -2.23981261e-01, -2.18843952e-01,
        2.11812109e-01,  3.45694095e-01,  2.30596781e-01, -3.41248244e-01,
       -2.96470553e-01,  

In [ ]:
from tqdm import tqdm

In [ ]:
b =  []
for text in tqdm(df['review'].values):
  b.append(remove_oov(text))

100%|██████████| 9980/9980 [02:39<00:00, 62.58it/s]


In [ ]:
b

[array([-3.13596785e-01,  5.93843162e-01,  8.47128630e-02, -7.49876872e-02,
         5.99066168e-03, -5.97489834e-01,  3.70680273e-01,  8.89328122e-01,
        -3.33735734e-01, -3.28930616e-01, -4.23450135e-02, -8.62238586e-01,
         4.03640233e-02,  2.34051451e-01,  1.99727938e-01, -4.28650379e-01,
        -1.74426422e-01, -4.86758143e-01, -2.37391010e-01, -1.08751523e+00,
         2.80952007e-01,  2.97674567e-01,  4.45537776e-01, -6.86053261e-02,
        -2.81059116e-01, -9.67835188e-02, -4.65696864e-02, -4.42711532e-01,
        -3.94105673e-01, -2.01032814e-02,  3.87424886e-01,  7.54530132e-02,
         2.84268856e-01, -6.20915174e-01, -1.35906100e-01,  4.00169164e-01,
         6.85982481e-02, -5.91216564e-01, -2.35666975e-01, -6.21338487e-01,
         2.72244122e-02, -4.36428428e-01, -1.44166082e-01,  4.78198566e-02,
         2.31442675e-01, -2.23596573e-01, -2.23981261e-01, -2.18843952e-01,
         2.11812109e-01,  3.45694095e-01,  2.30596781e-01, -3.41248244e-01,
        -2.9

In [ ]:
b= np.array(b)

In [ ]:
b

array([[-0.31359679,  0.59384316,  0.08471286, ..., -0.4324509 ,
         0.11488362, -0.18630618],
       [-0.2684854 ,  0.6048597 , -0.16039905, ..., -0.41734585,
         0.1006414 , -0.04173498],
       [-0.22928625,  0.38547227,  0.07833388, ..., -0.33036673,
        -0.04694499, -0.1718887 ],
       ...,
       [-0.24552467,  0.43645087, -0.15058973, ..., -0.31213915,
         0.17311741, -0.19853787],
       [-0.32821432,  0.46981066, -0.11247046, ..., -0.42380348,
         0.23875724, -0.04180256],
       [-0.4035943 ,  0.6858886 , -0.20935415, ..., -0.39178178,
        -0.0478555 , -0.21482392]], dtype=float32)

In [ ]:
b[0].shape

(100,)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lb = LabelEncoder()
y1 = lb.fit_transform(df['sentiment'])

In [ ]:
y1

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test = train_test_split(b, y1, test_size=0.2, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(x1_train,y1_train)
pred5 = rf.predict(x1_test)
accuracy_score(y1_test,pred5)

0.7900801603206413